In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, ec4c4a4c-3e32-4c0f-ae26-320e175193a7, 10, Finished, Available, Finished)

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

StatementMeta(, ec4c4a4c-3e32-4c0f-ae26-320e175193a7, 11, Finished, Available, Finished)

In [ ]:
# URL de base du repository GitHub
base_url = "https://raw.githubusercontent.com/gsoulat/data-training-fabric/main/eolienne/"

# Chemin vers la table Bronze
bronze_table_path = "abfss://Quentino@onelake.dfs.fabric.microsoft.com/LH_Bronze.Lakehouse/Tables/dbo/wind_power"



StatementMeta(, ec4c4a4c-3e32-4c0f-ae26-320e175193a7, 12, Finished, Available, Finished)

In [ ]:
# Vérifier que la table existe et afficher quelques statistiques
df_check = spark.read.format("delta").load(bronze_table_path)

print(f"📊 Nombre total de lignes : {df_check.count()}")
print(f"📅 Colonnes disponibles : {df_check.columns}")
print(f"🔍 Aperçu des 5 premières lignes :")
df_check.show(5)

StatementMeta(, ec4c4a4c-3e32-4c0f-ae26-320e175193a7, 13, Finished, Available, Finished)

📊 Nombre total de lignes : 864
📅 Colonnes disponibles : ['production_id', 'date', 'time', 'turbine_name', 'capacity', 'location_name', 'latitude', 'longitude', 'region', 'status', 'responsible_department', 'wind_speed', 'wind_direction', 'energy_produced']
🔍 Aperçu des 5 premières lignes :
+-------------+-------------------+--------+------------+--------+-------------+--------+---------+--------+------+----------------------+----------+--------------+---------------+
|production_id|               date|    time|turbine_name|capacity|location_name|latitude|longitude|  region|status|responsible_department|wind_speed|wind_direction|energy_produced|
+-------------+-------------------+--------+------------+--------+-------------+--------+---------+--------+------+----------------------+----------+--------------+---------------+
|         6697|2024-06-16 00:00:00|12-00-00|   Turbine A|    2200|   Location 1| 34.0522|-118.2437|Region A|Online|            Operations|  24.19474|            NW|  

In [ ]:
# Charger les données existantes depuis Bronze
df_spark = spark.read.format("delta").load(bronze_table_path)

# Convertir en Pandas pour manipulation plus facile
df_pandas = df_spark.toPandas()

print(f"📊 Données actuelles dans Bronze : {len(df_pandas)} lignes")

StatementMeta(, ec4c4a4c-3e32-4c0f-ae26-320e175193a7, 14, Finished, Available, Finished)

📊 Données actuelles dans Bronze : 864 lignes


In [ ]:
# Trouver la date la plus récente dans les données
most_recent_date = pd.to_datetime(df_pandas['date'], format="%Y%m%d").max()

# Calculer le jour suivant
next_date = (most_recent_date + timedelta(days=1)).strftime("%Y%m%d")

print(f"📅 Date la plus récente : {most_recent_date.strftime('%Y-%m-%d')}")
print(f"➡️  Prochaine date à charger : {next_date}")

StatementMeta(, ec4c4a4c-3e32-4c0f-ae26-320e175193a7, 15, Finished, Available, Finished)

📅 Date la plus récente : 2024-06-16
➡️  Prochaine date à charger : 20240617


In [ ]:
# Construire l'URL du fichier CSV
file_url = f"{base_url}{next_date}_wind_power_data.csv"
print(f"🌐 URL du fichier : {file_url}")

try:
    # Télécharger le CSV depuis GitHub
    df_pandas_new = pd.read_csv(file_url)
    
    # Convertir la colonne date en datetime
    df_pandas_new['date'] = pd.to_datetime(df_pandas_new['date'])
    
    print(f"✅ Nouvelles données téléchargées : {len(df_pandas_new)} lignes")
    print(f"📊 Aperçu des nouvelles données :")
    print(df_pandas_new.head())
    
except Exception as e:
    print(f"❌ Erreur lors du téléchargement : {e}")
    print(f"💡 Cela peut signifier que le fichier pour la date {next_date} n'existe pas encore.")

StatementMeta(, ec4c4a4c-3e32-4c0f-ae26-320e175193a7, 16, Finished, Available, Finished)

🌐 URL du fichier : https://raw.githubusercontent.com/gsoulat/data-training-fabric/main/eolienne/20240617_wind_power_data.csv
✅ Nouvelles données téléchargées : 432 lignes
📊 Aperçu des nouvelles données :
   production_id       date      time turbine_name  capacity location_name  \
0           6913 2024-06-17  00-00-00    Turbine A      2200    Location 1   
1           6914 2024-06-17  00-00-00    Turbine B      2000    Location 2   
2           6915 2024-06-17  00-00-00    Turbine C      2500    Location 3   
3           6916 2024-06-17  00-10-00    Turbine A      2200    Location 1   
4           6917 2024-06-17  00-10-00    Turbine B      2000    Location 2   

   latitude  longitude    region  status responsible_department  wind_speed  \
0   34.0522  -118.2437  Region A  Online             Operations     7.73222   
1   36.7783  -119.4179  Region B  Online             Operations    19.67376   
2   40.7128   -74.0060  Region C  Online             Operations    23.32255   
3   34.0522

In [ ]:
# Convertir le DataFrame Pandas en Spark DataFrame
df_spark_new = spark.createDataFrame(df_pandas_new, schema=df_spark.schema)

# Ajouter les nouvelles données à la table Bronze (mode append)
df_spark_new.write.format('delta').mode("append").save(bronze_table_path)

print("✅ Données ajoutées avec succès dans le Lakehouse Bronze")
print(f"📊 Total après ajout : {spark.read.format('delta').load(bronze_table_path).count()} lignes")

StatementMeta(, ec4c4a4c-3e32-4c0f-ae26-320e175193a7, 17, Finished, Available, Finished)

✅ Données ajoutées avec succès dans le Lakehouse Bronze
📊 Total après ajout : 1296 lignes
